In [1]:
from flask import Flask, render_template, request, jsonify
import pandas as pd
import threading
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Initialize Flask app
app = Flask(__name__)

# Dataset preprocessing and model setup
file_path = "evdataset.csv"  # Ensure this file is in the current directory
data = pd.read_csv(file_path)

# Define features and target
features = ['Acceleration 0 - 100 km/h', 'Top Speed', 'Battery Capacity', 'Seats', 'Drive']
target = 'Combined - Mild Weather'

# Preprocessing pipeline
numerical_features = ['Acceleration 0 - 100 km/h', 'Top Speed', 'Battery Capacity', 'Seats']
categorical_features = ['Drive']

numerical_transformer = StandardScaler()

# Make sure the OneHotEncoder can handle all the categories, including those not present in the dataset
categorical_transformer = OneHotEncoder(handle_unknown='ignore')  # Handle unknown categories gracefully

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Train a RandomForestRegressor model
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

X = data[features]
y = data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model_pipeline.fit(X_train, y_train)

# Flask routes
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Parse input data from form
        input_data = [
            float(request.form['acceleration']),
            float(request.form['top_speed']),
            float(request.form['battery_capacity']),
            int(request.form['seats']),
            request.form['drive']  # Drive type (FWD, RWD, AWD)
        ]
        
        # Create DataFrame for input
        input_df = pd.DataFrame([input_data], columns=features)
        
        # Predict the range using the trained model
        predicted_range = model_pipeline.predict(input_df)[0]
        
        return jsonify({
            'status': 'success',
            'predicted_range': f"{predicted_range:.2f} km"
        })
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)})

# Run Flask in a separate thread
def run_flask():
    app.run(port=5000)

thread = threading.Thread(target=run_flask)
thread.daemon = True
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
